# Machine Learning pour prédire la concentration horaire de polluants

Dans ce notebook, vous trouvez notre code d'entraînement et validation automatique d'algorithmes de Machine Learning pour prédire la concentration horaire de polluants à partir du jeu de données *Concentration horaire des polluants - Air ambiant - Lig'Air - Orléans Métropole*. <br>
On s'intéressera à une tâche de régression et à une tâche de classification sur ce même dataset.

**dataset:**https://www.data.gouv.fr/fr/datasets/concentration-horaire-des-polluants-air-ambiant-ligair-orleans-metropole/ <br>
**tâche 1:** Régression <br>
**target variable 1**:*valeur* 

**tâche 2:** Classification (Classification multi-classe) <br>
**target variable 2**:*nom_pollu* 

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split 
#pour l'entraînement automatique:
from supervised.automl import AutoML
#pour générer un rapport html des résultats:
import IPython
import markdown

## 1. Importation des données

On importe le dataset à partir de son url.

In [2]:
url = 'https://www.data.gouv.fr/fr/datasets/r/ce203343-6ed9-4fd3-b310-e553ae437f6d'
data_conc_pollu = pd.read_csv(url,sep=';')

## 2. Preprocessing et feature engineering

On élimine les données non exploitables (i.e. telles que *statut_valid* = 0) 

In [3]:
data_conc_pollu = data_conc_pollu[data_conc_pollu['statut_valid']==1]

On modifie le type de certaines variables et on crée des nouvelles variables.

In [4]:
#création des colonnes latitude et longitude
data_conc_pollu['lat'] = data_conc_pollu['geo_point_2d'].str.split(',').str[0].astype(float)
data_conc_pollu['long'] = data_conc_pollu['geo_point_2d'].str.split(',').str[1].astype(float)

#modification du type des variables catégorielles
categorical_var = ['nom_com','insee_com','nom_station','code_station','typologie','influence','nom_poll','id_poll_ue']
for c in categorical_var:
    data_conc_pollu[c] = data_conc_pollu[c].astype('category').cat.codes

#modification du type des variables en datetime
data_conc_pollu['date_debut'] = pd.to_datetime(data_conc_pollu['date_debut'],format='%Y-%m-%dT%H:%M:%S', utc=True)
data_conc_pollu['date_fin'] = pd.to_datetime(data_conc_pollu['date_fin'],format='%Y-%m-%dT%H:%M:%S', utc=True)

#création de nouvelles colonnes
data_conc_pollu['jour_semaine_debut'] = data_conc_pollu['date_debut'].dt.weekday
data_conc_pollu['jour_semaine_fin'] = data_conc_pollu['date_fin'].dt.weekday
data_conc_pollu['jour_debut'] = data_conc_pollu['date_debut'].dt.day
data_conc_pollu['jour_fin'] = data_conc_pollu['date_fin'].dt.day
data_conc_pollu['heure_debut'] = data_conc_pollu['date_debut'].dt.hour
data_conc_pollu['heure_fin'] = data_conc_pollu['date_fin'].dt.hour
data_conc_pollu['minute_debut'] = data_conc_pollu['date_debut'].dt.minute
data_conc_pollu['minute_fin'] = data_conc_pollu['date_fin'].dt.minute

#élimination de colonnes
data_conc_pollu = data_conc_pollu.drop(columns=['nom_dept','metrique','unite','code_epci','statut_valid','geo_shape','geo_point_2d','date_debut','date_fin'])

## 3. Régression

### 3.1. Train/Test splitting

On définit la variable réponse et les variables explicatives.
On partage le dataset en dataset d'entraînement et de test. Attention, AutoML nécessite d'un partage 75%/25%.

In [8]:
y_reg = data_conc_pollu['valeur'].values
X_reg = data_conc_pollu.drop(columns=['valeur'])

In [9]:
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(X_reg, y_reg, test_size=0.25, random_state=42)

### 3.2. AutoML

On entraîne, test et valide automatiquement plusieurs algorithmes de ML sur le dataset avec AutoML (plus d'infos ici: https://supervised.mljar.com/) et on génère un html des métriques et des paramètres du modèle.

In [ ]:
automl = AutoML(total_time_limit=5*60,mode='Explain',random_state=42)
#fit model
automl.fit(X_train_reg,y_train_reg)
#predictions
predictions = automl.predict(X_test_reg)
#generate report
automl.report()

## 4. Classification

### 4.1. Train/Test splitting
On définit la variable réponse et les variables explicatives. On partage le dataset en dataset d'entraînement et de test. Attention, AutoML nécessite d'un partage 75%/25%.

In [10]:
y_class = data_conc_pollu['nom_poll'].values
X_class = data_conc_pollu.drop(columns=['nom_poll','id_poll_ue'])

In [11]:
X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(X_class, y_class, test_size=0.25, random_state=42)

In [ ]:
automl = AutoML(total_time_limit=5*60,mode='Explain',random_state=42)
#fit model
automl.fit(X_train,y_train)
#predictions
predictions = automl.predict(X_test)
#generate html report
automl.report()